In [ ]:
from collections import Counter
from enum import Enum

In [ ]:
class HandType(Enum):
    FIVE_OF_A_KIND = 7
    FOUR_OF_A_KIND = 6
    FULL_HOUSE = 5
    THREE_OF_A_KIND = 4
    TWO_PAIR = 3
    PAIR = 2
    HIGH_CARD = 1

    def __lt__(self, other):
        if self.__class__ is other.__class__:
            return self.value < other.value
        return NotImplemented

In [ ]:
class Hand:
    def __init__(self, cards: str, bid: int, jokers: bool = False) -> None:
        self.cards = cards
        self.bid = bid
        self.jokers = jokers

    @property
    def hand_type(self) -> HandType:
        counts = Counter(self.cards)
        if self.jokers:
            # Add number of jokers to the most common card
            most_common = counts.most_common(1)[0][0]
            if most_common == "J":
                if len(counts) == 1:
                    # Special case: 5 jokers
                    return HandType.FIVE_OF_A_KIND
                most_common = counts.most_common(2)[1][0]
            counts[most_common] += counts["J"]
            del counts["J"]
        if len(counts) == 1:
            return HandType.FIVE_OF_A_KIND
        if len(counts) == 2:
            if 4 in counts.values():
                return HandType.FOUR_OF_A_KIND
            return HandType.FULL_HOUSE
        if len(counts) == 3:
            if 3 in counts.values():
                return HandType.THREE_OF_A_KIND
            return HandType.TWO_PAIR
        if len(counts) == 4:
            return HandType.PAIR
        return HandType.HIGH_CARD

    def __lt__(self, other: "Hand") -> bool:
        if self.hand_type == other.hand_type:
            for card1, card2 in zip(self.cards, other.cards):
                if self.card_value(card1) != other.card_value(card2):
                    return self.card_value(card1) < other.card_value(card2)
        return self.hand_type < other.hand_type

    def card_value(self, card: str) -> int:
        match card:
            case "A":
                return 14
            case "K":
                return 13
            case "Q":
                return 12
            case "J":
                if not self.jokers:
                    return 11
                return 1
            case "T":
                return 10
            case _:
                return int(card)

    @classmethod
    def from_string(cls, string: str, jokers: bool = False) -> "Hand":
        cards, bid = string.split()
        return cls(cards=cards, bid=int(bid), jokers=jokers)

    def __repr__(self) -> str:
        return f"Hand(cards={self.cards}, bid={self.bid}, jokers={self.jokers})"

# Part 1


In [ ]:
hands = []
with open("day07_input.txt") as file:
    for line in file:
        hands.append(Hand.from_string(line.strip()))

print(
    "Answer:", sum(rank * card.bid for rank, card in enumerate(sorted(hands), start=1))
)

# Part 2


In [ ]:
hands = []
with open("day07_input.txt") as file:
    for line in file:
        hands.append(Hand.from_string(line.strip(), jokers=True))

print(
    "Answer:", sum(rank * card.bid for rank, card in enumerate(sorted(hands), start=1))
)